In [ ]:
import emcee
import corner
import speclite as speclite; from speclite import filters
from tqdm import tqdm
from matplotlib import cm
from matplotlib.artist import Artist
from chromatic import *
from scipy.optimize import minimize
from scipy.optimize import curve_fit
from PyAstronomy import pyasl
from specutils.spectra import Spectrum1D, SpectralRegion
from specutils.fitting import fit_generic_continuum

transmission_data = np.loadtxt('../data/transmission_comp.txt')
mol_wave = transmission_data[:,0] * u.nm
mol_data = transmission_data[:,1]

In [ ]:
bandpass=np.linspace(3800.,10000.,400)*u.angstrom

#response functions

gp_data = np.loadtxt('../data/filter_curves/SDSS.gp.txt')
gp_response = gp_data[:,1]
gp_response[gp_response<0] = 0
gp_response[-1] = 0
rp_data = np.loadtxt('../data/filter_curves/SDSS.rp.txt')
rp_response = rp_data[:,1]
rp_response[rp_response<0] = 0
rp_response[-1] = 0
ip_data = np.loadtxt('../data/filter_curves/SDSS.ip.txt')
ip_response = ip_data[:,1]
ip_response[ip_response<0] = 0
ip_response[-1] = 0
LCO_gp = speclite.filters.FilterResponse(
    wavelength = gp_data[:,0]*10 * u.Angstrom,
    response = gp_response, meta=dict(group_name='sdss', band_name='gp'))
LCO_rp = speclite.filters.FilterResponse(
    wavelength = rp_data[:,0]*10 * u.Angstrom,
    response = rp_response, meta=dict(group_name='sdss', band_name='rp'))
LCO_ip = speclite.filters.FilterResponse(
    wavelength = ip_data[:,0]*10 * u.Angstrom,
    response = ip_response, meta=dict(group_name='sdss', band_name='ip'))

sdss_responses = speclite.filters.load_filters('sdss-gp','sdss-rp','sdss-ip')
response_g = sdss_responses[0].interpolator(bandpass)
response_r = sdss_responses[1].interpolator(bandpass)
response_i = sdss_responses[2].interpolator(bandpass)

## Spectrum normalization functions

In [ ]:
def normalize_nres(data_flux,wavelength=None,**kwargs):
    
    flux = data_flux * u.erg
    
    _m = flux/np.nanmedian(flux)
    spectrum = Spectrum1D(flux=_m, spectral_axis=wavelength)
    with warnings.catch_warnings():  # Ignore warnings
        warnings.simplefilter('ignore')
        g1_fit = fit_generic_continuum(spectrum)
        continuum_fit = g1_fit(wavelength)
        
    normed_flux = _m/continuum_fit
    
    return normed_flux

In [ ]:
def normalized_1T_PHOENIX(Tspec=3650,wavelength=None,**kwargs):
        
    _model = get_phoenix_photons(temperature=float(Tspec),wavelength=wavelength,
                                 logg=4.4, metallicity=0.0)
    
    _m = _model[1]/np.nanmedian(_model[1])
    spectrum = Spectrum1D(flux=_m, spectral_axis=_model[0])
    with warnings.catch_warnings():  # Ignore warnings
        warnings.simplefilter('ignore')
        g1_fit = fit_generic_continuum(spectrum)
        continuum_fit = g1_fit(_model[0])
        
    normed_model = _m/continuum_fit
    
    return normed_model

In [ ]:
def normalized_2T_PHOENIX(spot_params=[0.35,0.07,2500.,4000.],wavelength = None,
                          spotspec=None,ambspec=None, **kwargs):
    
    f_spot,df_spot,T_spot, T_amb = spot_params
    
    _m = f_spot*spotspec[1] + (1.-f_spot)*ambspec[1]
    normed_spec = _m/np.nanmedian(_m)
    spectrum = Spectrum1D(flux=normed_spec, spectral_axis=wavelength)
    with warnings.catch_warnings():  # Ignore warnings
        warnings.simplefilter('ignore')
        g1_fit = fit_generic_continuum(spectrum)
        continuum_fit = g1_fit(wavelength)
    model = normed_spec/continuum_fit
    
    return model

## Photometric variability model

In [ ]:
def photometric_variability_model(parameters=None,plot=False,
                                  samples_exist=False,samples=None,
                                  label=None,title=None,
                                  **kwargs):
    
    f_spot,df_spot,T_spot,T_amb = parameters
    bandpass=np.linspace(3800.,10000.,400)*u.angstrom

    spotflux = get_phoenix_photons(temperature=int(T_spot), wavelength = bandpass,logg=4.4, metallicity=0.0)[1]
    ambflux = get_phoenix_photons(temperature=int(T_amb), wavelength = bandpass,logg=4.4, metallicity=0.0)[1]
    this_model_spectrum = f_spot*spotflux + (1.0-f_spot)*ambflux
                     
    d_lambda = (bandpass[1]-bandpass[0])
    contrast = 1.-(spotflux/ambflux)
    ds_over_s = -df_spot * ( contrast / ( 1.-f_spot * contrast ) )
    semi_amplitude = np.abs(ds_over_s)

    numerator = np.nansum(semi_amplitude*this_model_spectrum*response_g*d_lambda)
    denominator = np.nansum(this_model_spectrum*response_g*d_lambda)
    modelgp = numerator/denominator

    numerator = np.nansum(semi_amplitude*this_model_spectrum*response_r*d_lambda)
    denominator = np.nansum(this_model_spectrum*response_r*d_lambda)
    modelrp = numerator/denominator

    numerator = np.nansum(semi_amplitude*this_model_spectrum*response_i*d_lambda)
    denominator = np.nansum(this_model_spectrum*response_i*d_lambda)
    modelip = numerator/denominator

    if visit == 'F21':
        model = np.array([modelgp,modelrp,modelip])
        model_coords = [4750,6200,7550]
        w_err = [500,400,500]
        phot_data = np.array([0.075,0.075,0.05])
        phot_errs = np.array([0.005,0.005,0.006])
        
    if visit == 'S22':
        model = np.array([modelgp,modelrp])
        model_coords = [4750,6200]
        w_err = [500,400]
        phot_data = np.array([0.076,0.076])
        phot_errs = np.array([0.004,0.004])
        
    chisq = np.nansum((phot_data - model)**2./(phot_errs)**2.)
    err_weight = np.nansum(1./np.sqrt(2.*np.pi*(phot_errs)))
    ln_like = (err_weight - 0.5*chisq)
    
    return model, ln_like

## Spectral model

In [ ]:
def average_spectrum_model(parameters=[0.35,0.07,3000.,3900.],
                           data_flux=None, data_err=None,
                           wavelength=None, **kwargs):
    
    S_spot = get_phoenix_photons(temperature=int(parameters[2]),wavelength=wavelength,
                                    logg=4.4,metallicity=0.0)
    S_amb = get_phoenix_photons(temperature=int(parameters[3]),wavelength=wavelength,
                                   logg=4.4,metallicity=0.0)

    model = normalized_2T_PHOENIX(spot_params=parameters,wavelength = wavelength,
                                  spotspec=S_spot,ambspec=S_amb)
    
    chisq = np.nansum((data_flux-model)**2/(data_err**2))
    err_weight = np.nansum(1./np.sqrt(2.*np.pi*(data_err)))
    ln_like = (err_weight - 0.5*chisq)
    
    return model, ln_like

In [ ]:
NRES_Spectra = {
    
    "F21" : {
        
        "53" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "54" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "55" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "56" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "57" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "58" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "59" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "60" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "61" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "62" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "63" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "64" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "65" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "66" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "67" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "68" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "69" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        
        "70" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "71" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "72" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "73" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "74" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "75" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        
        "76" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "77" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "78" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "79" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "80" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "81" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "82" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "83" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        }
    },
    
    "S22" : {
        "53" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "54" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "55" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "56" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "57" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "58" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "59" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "60" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "61" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "62" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "63" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "64" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "65" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "66" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "67" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "68" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "69" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        
        "70" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "71" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "72" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "73" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "74" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "75" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        
        "76" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "77" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "78" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "79" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "80" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "81" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "82" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        },
        "83" : {
            "w" : None,"f" : None,"e" : None, "ref_w" : None,
        }  
    }
}

In [ ]:
all_orders = np.array([53,54,55,56,57,58,59,
                        60,61,62,63,64,65,66,67,68,69,
                        70,71,72,73,74,75,76,77,78,79,
                        80,81,82,83])

for visit in tqdm(['F21','S22']):
    for order in tqdm(all_orders):
        print(visit,order)
        _r = read_rainbow(f"../data/rainbows/{visit}_{order}_clipped.rainbow.npy")
        _ref_r = read_rainbow(f"../data/rainbows/{visit}_{order}_original.rainbow.npy")
        ref_r = _ref_r.trim()
        ref_w = ref_r.wavelength.value
        dw = _r.wavelength
        
        _1dspec = _r.get_average_spectrum()
        good_values = np.isfinite(_1dspec)
        
        data_wave = dw[good_values]
        print(data_wave[3]-data_wave[0],ref_r.wavelength.unit)
        
        nres_avg_1dspec = normalize_nres(_1dspec[good_values], wavelength=data_wave)
        nres_avg_1derr = np.nanmedian(_r.uncertainty[good_values],axis=1)/np.sqrt(len(_r.timelike['time']))
        template_spec = normalized_1T_PHOENIX(Tspec=3650,wavelength=data_wave)
        rchisq = np.nansum((template_spec-nres_avg_1dspec)**2/(nres_avg_1derr)**2) / (len(data_wave)-1)
        if rchisq > 1.0:
            nres_avg_1derr = nres_avg_1derr * np.sqrt(rchisq)
            
        NRES_Spectra[f'{visit}'][f'{order}']['w'] = data_wave
        NRES_Spectra[f'{visit}'][f'{order}']['ref_w'] = ref_w
        NRES_Spectra[f'{visit}'][f'{order}']['f'] = nres_avg_1dspec
        NRES_Spectra[f'{visit}'][f'{order}']['e'] = nres_avg_1derr*1.25

## Define the log probability

In [ ]:
def lnprob(parameters=None,**kwargs):

    f_spot,df_spot,T_spot,T_amb = parameters

    if (0.0<=f_spot<=0.5) and (0.<df_spot<f_spot) and (2300.0<=T_spot<=7000.0) and (2300.0<=T_amb<=7000.0):

        ln_like=0.0
        
        "Teff likelihood"
        if do_Teff:
            Teff_model = (f_spot*(T_spot**4.) + (1.-f_spot)*(T_amb**4.))**(1./4.)    
            chisq_Teff = (3650. - Teff_model)**2./(100.)**2.
            err_weight_Teff = 1./np.sqrt(2.*np.pi*(100.))
            
            ln_like += (err_weight_Teff - 0.5*chisq_Teff)
            
        "Photometric variability likelihood"
        if do_Photometry:
            ln_like_phot = photometric_variability_model(parameters=parameters,visit=visit)[1]
            ln_like += ln_like_phot
            
        "Combined phot variability likelihood"
        if do_Combined_Photometry:
            ln_like_phot_F21 = photometric_variability_model(parameters=parameters,visit='F21')[1]
            ln_like_phot_S22 = photometric_variability_model(parameters=parameters,visit='S22')[1]
            ln_like += ln_like_phot_F21
            ln_like += ln_like_phot_S22
            
        "Visit-specific spectral likelihood"
        if do_AvgSpec:
            data_wave = NRES_Spectra[f'{visit}'][f'{order}']['w']
            data_spec = NRES_Spectra[f'{visit}'][f'{order}']['f']
            data_err = NRES_Spectra[f'{visit}'][f'{order}']['e']

            ln_like_spec = average_spectrum_model(parameters=parameters, wavelength=data_wave,
                                                   data_flux=data_spec, data_err=data_err)[1]
            ln_like += ln_like_spec

        "Combined visit likelihood"
        if do_CombinedSpec:
            
            F21_wave = NRES_Spectra['F21'][f'{order}']['w']
            F21_spec = NRES_Spectra['F21'][f'{order}']['f']
            F21_err = NRES_Spectra['F21'][f'{order}']['e']

            S22_wave = NRES_Spectra['S22'][f'{order}']['w']
            S22_spec = NRES_Spectra['S22'][f'{order}']['f']
            S22_err = NRES_Spectra['S22'][f'{order}']['e']

            ln_like_F21 = average_spectrum_model(parameters=parameters, wavelength=F21_wave,
                                                   data_flux=F21_spec, data_err=F21_err)[1]
            ln_like_S22 = average_spectrum_model(parameters=parameters,wavelength=S22_wave,
                                                   data_flux=S22_spec,data_err=S22_err)[1]
            ln_like += ln_like_F21
            ln_like += ln_like_S22 

    else:
        ln_like = -np.inf

    return ln_like

## Make a wrapper for the sampler

In [ ]:
def do_mcmc(label='oops you didnt label your samples :sadface:',
            nsteps=100,burnin=25,ndim=4,nwalkers=100,**kwargs):
    
    # these are initial parameters
    fspot_init = np.random.uniform(0.05, 0.35, nwalkers)
    dfspot_init = np.random.uniform(0.01, 0.1, nwalkers)
    Tspot_init = np.random.uniform(2800, 3300, nwalkers)
    Tamb_init = np.random.uniform(3800, 4000, nwalkers)
    p0 = np.transpose([fspot_init, dfspot_init, Tspot_init, Tamb_init])

    # set up file saving for the samples when finished
    filename = f"../data/samples/{label}.h5"
    backend = emcee.backends.HDFBackend(filename)
    backend.reset(nwalkers, ndim)
    # Initialize and run the sampler
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, backend=backend)
    result = sampler.run_mcmc(p0, nsteps,store=True)
    samples = sampler.chain[:, burnin:, :].reshape((-1, ndim)).T

    for i in range(len(samples)):
        tau_f = emcee.autocorr.integrated_time(samples[i])
        print('(Nsteps-burnin)*nwalkers/tau=',(nsteps-burnin)*nwalkers/tau_f)
    
    return samples

## Run Teff + Photometry Model

In [ ]:
# nsteps = 1000
# modeltype = 'individual_Teff_Phot'
# visits = ['F21','S22']
# order = 52
# do_Teff = True
# do_Photometry = True
# do_AvgSpec = False
# do_CombinedSpec = False
# do_Combined_Photometry = False

# for visit in visits:
#     # Run the MCMC
#     print('beginning MCMC..')
#     samples = do_mcmc(label = f'{visit}_{order}_{modeltype}_{nsteps}steps',
#                       nsteps = nsteps, burnin = int(0.25*nsteps))

## Run Teff + Spec Model

In [ ]:
modeltype = 'individual_orders_Teff_Spec'
visits = ['F21','S22','combined']
nsteps = 1000
orders_to_model = np.arange(53,84)
# bad_array = np.array([56,57,64,65,67,67,68,74,78,79,69,73,75,80])
bad_array = np.array([])

do_Teff = True
do_Photometry=False
do_AvgSpec=True
do_CombinedSpec=False
do_Combined_Photometry = False
# for visit in ['F21','S22']:
#     for order in orders_to_model:
#         nsteps = 1000
#         bad_flag = (bad_array == order)
#         if np.any(bad_flag):
#             nsteps = 500

#         # Run the MCMC
#         print('beginning MCMC..')
#         samples = do_mcmc(label = f'{visit}_{order}_{modeltype}_1000steps',
#                           nsteps = nsteps, burnin = int(0.25*nsteps))

do_CombinedSpec=True
do_AvgSpec=False
for visit in ['combined']:
    for order in tqdm(orders_to_model):
        nsteps = 500
        bad_flag = (bad_array == order)
        if np.any(bad_flag):
            nsteps = 500
            
        # Run the MCMC
        print('beginning MCMC..')
        samples = do_mcmc(label = f'{visit}_{order}_{modeltype}_500steps',
                          nsteps = nsteps, burnin = int(0.25*nsteps))

## Run Teff + Photometry + Spectra Model

In [ ]:
modeltype = 'individual_orders_Teff_Phot_Spec'
orders_to_model = all_orders
# bad_array = np.array([56,57,64,65,66,67,68,74,78,79,69,73,75,80])

do_Teff = True
do_Photometry = True
do_AvgSpec = True
do_CombinedSpec=False
do_Combined_Photometry=False
for visit in ['F21','S22']:
    for order in orders_to_model:
        nsteps = 2000
        bad_flag = (bad_array == order)
        if np.any(bad_flag):
            nsteps = 500

        # Run the MCMC
        print('beginning MCMC..')
        samples = do_mcmc(label = f'{visit}_{order}_{modeltype}_2000steps',
                          nsteps = nsteps, burnin = int(0.25*nsteps))
        
do_Teff = True
do_Photometry=False
do_AvgSpec=False
do_CombinedSpec = True
do_Combined_Photometry = True
for visit in ['combined']:
    for order in tqdm(orders_to_model):
        nsteps = 2000
        bad_flag = (bad_array == order)
        if np.any(bad_flag):
            nsteps = 500

        # Run the MCMC
        print('beginning MCMC..')
        samples = do_mcmc(label = f'{visit}_{order}_{modeltype}_2000steps',
                          nsteps = nsteps, burnin = int(0.25*nsteps))